<a href="https://colab.research.google.com/github/Bumper-Car/Vroomie_AI/blob/main/Lane_Deviation_and_Cut_In_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.chdir('/content/drive/MyDrive/Vroomie/inference')

In [3]:
!pip install ultralytics

In [4]:
!mkdir -p yolov8_pretrained
!wget -O yolov8_pretrained/coco128.txt https://raw.githubusercontent.com/ultralytics/yolov5/master/data/coco128.yaml

--2025-06-03 14:02:04--  https://raw.githubusercontent.com/ultralytics/yolov5/master/data/coco128.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1912 (1.9K) [text/plain]
Saving to: ‘yolov8_pretrained/coco128.txt’

yolov8_pretrained/c 100%[===================>]   1.87K  --.-KB/s    in 0s      

2025-06-03 14:02:04 (9.37 MB/s) - ‘yolov8_pretrained/coco128.txt’ saved [1912/1912]



In [5]:
!pip install deep-sort-realtime

In [6]:
!pip install fastapi
!pip install pyngrok
!pip install nest_asyncio
!pip install uvicorn

In [7]:
import math
import os
import threading

import cv2
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init
from torchvision import transforms

from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

from google.colab.patches import cv2_imshow
from IPython.display import clear_output

import asyncio
import nest_asyncio
import websockets
import base64
from datetime import datetime

from scipy.ndimage import gaussian_filter1d
from cuml.cluster import DBSCAN as cuDBSCAN
import cupy as cp
import cudf
import json
import requests

from fastapi import WebSocket
import json

In [8]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(DEVICE)

cuda:0


# Model

In [9]:
class InitialBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(InitialBlock, self).__init__()
        self.input_channel = in_ch
        self.conv_channel = out_ch - in_ch

        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch - in_ch, kernel_size = 3, stride = 2, padding=1),
            nn.BatchNorm2d(out_ch - in_ch),
            nn.PReLU()
        )
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        conv_branch = self.conv(x)
        maxp_branch = self.maxpool(x)
        return torch.cat([conv_branch, maxp_branch], 1)

In [10]:
class BottleneckModule(nn.Module):
    def __init__(self, in_ch, out_ch, module_type, padding = 1, dilated = 0, asymmetric = 5, dropout_prob = 0):
        super(BottleneckModule, self).__init__()
        self.input_channel = in_ch
        self.activate = nn.PReLU()

        self.module_type = module_type
        if self.module_type == 'downsampling':
            self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
            self.conv = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, kernel_size = 2, stride = 2),
                nn.BatchNorm2d(out_ch),
                nn.PReLU(),
                nn.Conv2d(out_ch, out_ch, kernel_size = 3, stride=1, padding=padding),
                nn.BatchNorm2d(out_ch),
                nn.PReLU(),
                nn.Conv2d(out_ch, out_ch, kernel_size = 1),
                nn.BatchNorm2d(out_ch),
                nn.PReLU(),
                nn.Dropout2d(p=dropout_prob)
            )
        elif self.module_type == 'upsampling':
            self.maxunpool = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, kernel_size = 1),
                nn.BatchNorm2d(out_ch),
                nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)    # Use upsample instead of maxunpooling
            )

            self.conv = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, kernel_size = 1),
                nn.BatchNorm2d(out_ch),
                nn.PReLU(),
                nn.ConvTranspose2d(out_ch, out_ch, kernel_size=2, stride=2),
                nn.BatchNorm2d(out_ch),
                nn.PReLU(),
                nn.Conv2d(out_ch, out_ch, kernel_size = 1),
                nn.BatchNorm2d(out_ch),
                nn.PReLU(),
                nn.Dropout2d(p=dropout_prob)
            )
        elif self.module_type == 'regular':
            self.conv = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, kernel_size = 1),
                nn.BatchNorm2d(out_ch),
                nn.PReLU(),
                nn.Conv2d(out_ch, out_ch, kernel_size = 3, stride=1, padding=padding),
                nn.BatchNorm2d(out_ch),
                nn.PReLU(),
                nn.Conv2d(out_ch, out_ch, kernel_size = 1),
                nn.BatchNorm2d(out_ch),
                nn.PReLU(),
                nn.Dropout2d(p=dropout_prob)
            )
        elif self.module_type == 'asymmetric':
            self.conv = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, kernel_size = 1),
                nn.BatchNorm2d(out_ch),
                nn.PReLU(),
                nn.Conv2d(out_ch, out_ch, (asymmetric, 1), stride=1, padding=(padding, 0)),
                nn.Conv2d(out_ch, out_ch, (1, asymmetric), stride=1, padding=(0, padding)),
                nn.BatchNorm2d(out_ch),
                nn.PReLU(),
                nn.Conv2d(out_ch, out_ch, kernel_size = 1),
                nn.BatchNorm2d(out_ch),
                nn.PReLU(),
                nn.Dropout2d(p=dropout_prob)
            )
        elif self.module_type == 'dilated':
            self.conv = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, kernel_size = 1),
                nn.BatchNorm2d(out_ch),
                nn.PReLU(),
                nn.Conv2d(out_ch, out_ch, kernel_size = 3, stride=1, padding=padding, dilation=dilated),
                nn.BatchNorm2d(out_ch),
                nn.PReLU(),
                nn.Conv2d(out_ch, out_ch, kernel_size = 1),
                nn.BatchNorm2d(out_ch),
                nn.PReLU(),
                nn.Dropout2d(p=dropout_prob)
            )
        else:
            raise("Module Type error")

    def forward(self, x):
        if self.module_type == 'downsampling':
            conv_branch = self.conv(x)
            maxp_branch = self.maxpool(x)
            bs, conv_ch, h, w = conv_branch.size()
            maxp_ch = maxp_branch.size()[1]
            padding = torch.zeros(bs, conv_ch - maxp_ch, h, w).to(DEVICE)

            maxp_branch = torch.cat([maxp_branch, padding], 1).to(DEVICE)
            output = maxp_branch + conv_branch
        elif self.module_type == 'upsampling':
            conv_branch = self.conv(x)
            maxunp_branch = self.maxunpool(x)
            output = maxunp_branch + conv_branch
        else:
            output = self.conv(x) + x

        return self.activate(output)

In [11]:
def weights_init_kaiming(m):
    classname = m.__class__.__name__
    #print(classname)
    if classname.find('Conv') != -1:
        init.kaiming_normal_(m.weight.data, a=0, mode='fan_in')
    elif classname.find('Linear') != -1:
        init.kaiming_normal_(m.weight.data, a=0, mode='fan_in')
    elif classname.find('BatchNorm') != -1:
        init.normal_(m.weight.data, 1.0, 0.02)
        init.constant_(m.bias.data, 0.0)

In [12]:
class ENet_Encoder(nn.Module):

    def __init__(self, in_ch=3, dropout_prob=0):
        super(ENet_Encoder, self).__init__()

        # Encoder

        self.initial_block = InitialBlock(in_ch, 16)

        self.bottleneck1_0 = BottleneckModule(16, 64, module_type = 'downsampling', padding = 1, dropout_prob = dropout_prob)
        self.bottleneck1_1 = BottleneckModule(64, 64, module_type = 'regular', padding = 1, dropout_prob = dropout_prob)
        self.bottleneck1_2 = BottleneckModule(64, 64, module_type = 'regular', padding = 1, dropout_prob = dropout_prob)
        self.bottleneck1_3 = BottleneckModule(64, 64, module_type = 'regular', padding = 1, dropout_prob = dropout_prob)
        self.bottleneck1_4 = BottleneckModule(64, 64, module_type = 'regular', padding = 1, dropout_prob = dropout_prob)

        self.bottleneck2_0 = BottleneckModule(64, 128, module_type = 'downsampling', padding = 1, dropout_prob = dropout_prob)
        self.bottleneck2_1 = BottleneckModule(128, 128, module_type = 'regular', padding = 1, dropout_prob = dropout_prob)
        self.bottleneck2_2 = BottleneckModule(128, 128, module_type = 'dilated', padding = 2, dilated = 2, dropout_prob = dropout_prob)
        self.bottleneck2_3 = BottleneckModule(128, 128, module_type = 'asymmetric', padding = 2, asymmetric=5, dropout_prob = dropout_prob)
        self.bottleneck2_4 = BottleneckModule(128, 128, module_type = 'dilated', padding = 4, dilated = 4, dropout_prob = dropout_prob)
        self.bottleneck2_5 = BottleneckModule(128, 128, module_type = 'regular', padding = 1, dropout_prob = dropout_prob)
        self.bottleneck2_6 = BottleneckModule(128, 128, module_type = 'dilated', padding = 8, dilated = 8, dropout_prob = dropout_prob)
        self.bottleneck2_7 = BottleneckModule(128, 128, module_type = 'asymmetric', padding = 2, asymmetric=5, dropout_prob = dropout_prob)
        self.bottleneck2_8 = BottleneckModule(128, 128, module_type = 'dilated', padding = 16, dilated = 16, dropout_prob = dropout_prob)

        self.bottleneck3_0 = BottleneckModule(128, 128, module_type = 'regular', padding = 1, dropout_prob = dropout_prob)
        self.bottleneck3_1 = BottleneckModule(128, 128, module_type = 'dilated', padding = 2, dilated = 2, dropout_prob = dropout_prob)
        self.bottleneck3_2 = BottleneckModule(128, 128, module_type = 'asymmetric', padding = 2, asymmetric=5, dropout_prob = dropout_prob)
        self.bottleneck3_3 = BottleneckModule(128, 128, module_type = 'dilated', padding = 4, dilated = 4, dropout_prob = dropout_prob)
        self.bottleneck3_4 = BottleneckModule(128, 128, module_type = 'regular', padding = 1, dropout_prob = dropout_prob)
        self.bottleneck3_5 = BottleneckModule(128, 128, module_type = 'dilated', padding = 8, dilated = 8, dropout_prob = dropout_prob)
        self.bottleneck3_6 = BottleneckModule(128, 128, module_type = 'asymmetric', padding = 2, asymmetric=5, dropout_prob = dropout_prob)
        self.bottleneck3_7 = BottleneckModule(128, 128, module_type = 'dilated', padding = 16, dilated = 16, dropout_prob = dropout_prob)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                weights_init_kaiming(m)
            elif isinstance(m, nn.BatchNorm2d):
                weights_init_kaiming(m)

    def forward(self, x):
        x = self.initial_block(x)

        x = self.bottleneck1_0(x)
        x = self.bottleneck1_1(x)
        x = self.bottleneck1_2(x)
        x = self.bottleneck1_3(x)
        x = self.bottleneck1_4(x)

        x = self.bottleneck2_0(x)
        x = self.bottleneck2_1(x)
        x = self.bottleneck2_2(x)
        x = self.bottleneck2_3(x)
        x = self.bottleneck2_4(x)
        x = self.bottleneck2_5(x)
        x = self.bottleneck2_6(x)
        x = self.bottleneck2_7(x)
        x = self.bottleneck2_8(x)

        x = self.bottleneck3_0(x)
        x = self.bottleneck3_1(x)
        x = self.bottleneck3_2(x)
        x = self.bottleneck3_3(x)
        x = self.bottleneck3_4(x)
        x = self.bottleneck3_5(x)
        x = self.bottleneck3_6(x)
        x = self.bottleneck3_7(x)

        return x

In [13]:
class ENet_Decoder(nn.Module):

    def __init__(self, out_ch=1, dropout_prob=0):
        super(ENet_Decoder, self).__init__()


        self.bottleneck4_0 = BottleneckModule(128, 64, module_type = 'upsampling', padding = 1, dropout_prob = dropout_prob)
        self.bottleneck4_1 = BottleneckModule(64, 64, module_type = 'regular', padding = 1, dropout_prob = dropout_prob)
        self.bottleneck4_2 = BottleneckModule(64, 64, module_type = 'regular', padding = 1, dropout_prob = dropout_prob)

        self.bottleneck5_0 = BottleneckModule(64, 16, module_type = 'upsampling', padding = 1, dropout_prob = dropout_prob)
        self.bottleneck5_1 = BottleneckModule(16, 16, module_type = 'regular', padding = 1, dropout_prob = dropout_prob)

        self.fullconv = nn.ConvTranspose2d(16, out_ch, kernel_size=2, stride=2)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                weights_init_kaiming(m)
            elif isinstance(m, nn.BatchNorm2d):
                weights_init_kaiming(m)

    def forward(self, x):

        x = self.bottleneck4_0(x)
        x = self.bottleneck4_1(x)
        x = self.bottleneck4_2(x)

        x = self.bottleneck5_0(x)
        x = self.bottleneck5_1(x)

        x = self.fullconv(x)

        return x

In [14]:
class LaneNet(nn.Module):
    def __init__(self, in_ch = 3, arch="ENet", output_size=(270, 480)):
        super(LaneNet, self).__init__()
        # no of instances for segmentation
        self.no_of_instances = 3  # if you want to output RGB instance map, it should be 3.
        print("Use {} as backbone".format(arch))
        self._arch = arch
        self.output_size = output_size

        self._encoder = ENet_Encoder(in_ch)
        self._encoder.to(DEVICE)

        self._decoder_binary = ENet_Decoder(2)
        self._decoder_instance = ENet_Decoder(self.no_of_instances)
        self._decoder_binary.to(DEVICE)
        self._decoder_instance.to(DEVICE)

        self.relu = nn.ReLU().to(DEVICE)
        self.sigmoid = nn.Sigmoid().to(DEVICE)

    def forward(self, input_tensor):
        c = self._encoder(input_tensor)
        binary = self._decoder_binary(c)
        instance = self._decoder_instance(c)

        binary = F.interpolate(binary, size=self.output_size, mode='bilinear', align_corners=True)
        instance = F.interpolate(instance, size=self.output_size, mode='bilinear', align_corners=True)

        binary_seg_ret = torch.argmax(F.softmax(binary, dim=1), dim=1, keepdim=True)

        pix_embedding = self.sigmoid(instance)

        return {
            'instance_seg_logits': pix_embedding,
            'binary_seg_pred': binary_seg_ret,
            'binary_seg_logits': binary
        }

# Car Detection

In [15]:
# 설정값
CONFIDENCE_THRESHOLD = 0.65
FRAME_SKIP = 1
(resize_height, resize_width) = (270, 480)
GREEN = (0, 255, 0)
WHITE = (255, 255, 255)
MAGENTA = (255, 0, 255)

In [16]:
# 클래스 목록
with open('./yolov8_pretrained/coco128.txt', 'r') as f:
    class_list = f.read().splitlines()

# YOLO 모델 로드
yolo_model = YOLO('./yolov8_pretrained/yolov8l.pt')
tracker = DeepSort(max_age=50, nms_max_overlap=0.5)

# LaneNet 모델 세팅
data_transform = transforms.Compose([
    transforms.Resize((resize_height, resize_width)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

In [17]:
# LaneNet 가중치 로드
model_path = '/content/drive/MyDrive/Vroomie/model/best_model.pth'
lanenet_model = LaneNet()
state_dict = torch.load(model_path, map_location=DEVICE)
lanenet_model.load_state_dict(state_dict)
lanenet_model.eval()
lanenet_model.to(DEVICE)

Use ENet as backbone


LaneNet(
  (_encoder): ENet_Encoder(
    (initial_block): InitialBlock(
      (conv): Sequential(
        (0): Conv2d(3, 13, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (1): BatchNorm2d(13, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): PReLU(num_parameters=1)
      )
      (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (bottleneck1_0): BottleneckModule(
      (activate): PReLU(num_parameters=1)
      (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (conv): Sequential(
        (0): Conv2d(16, 64, kernel_size=(2, 2), stride=(2, 2))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): PReLU(num_parameters=1)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): PReLU(num_p

In [18]:
import cuml

In [19]:
def extrapolate_lanes_from_instance(instance_pred, threshold=30):
    """
    직선 피팅을 통해 instance_pred 기반 확장 차선 마스크 생성

    Parameters:
    - instance_pred: (3, H, W)
    - threshold: float - 임베딩 평균값 threshold (차선 후보 픽셀 선택 기준)

    Returns:
    - extended_instance: np.ndarray (3, H, W) - 확장된 임베딩 마스크
    """

    C, H, W = instance_pred.shape
    instance_map = np.mean(instance_pred, axis=0)

    # 이진 마스크로 후보 픽셀 추출
    mask = instance_map > threshold

    # 차선 픽셀 좌표 추출 (y, x)
    coords = np.column_stack(np.where(mask))

    print(len(coords))

    if len(coords) == 0:
        return np.zeros((3, H, W), dtype=np.float32)

    ys, xs = coords[:, 0], coords[:, 1]
    embeddings = instance_pred[:, ys, xs].T

    # cupy 배열로 변환
    embeddings_gpu = cp.asarray(embeddings)

    # cuML DBSCAN 실행
    db = cuDBSCAN(eps=6, min_samples=30, max_mbytes_per_batch=12000)
    labels_gpu = db.fit_predict(embeddings_gpu)

    # GPU에서 CPU로 label 가져오기
    labels = cp.asnumpy(labels_gpu)
    valid_labels = set(labels) - {-1}

    # valid_labels = set(labels) - {-1}
    extended_instance = np.zeros((3, H, W), dtype=np.float32)

    for label in valid_labels:
        cluster_coords = coords[labels == label]
        if len(cluster_coords) < 10:
            continue

        ys = cluster_coords[:, 0]
        xs = cluster_coords[:, 1]

        try:
            coeffs = np.polyfit(xs, ys, deg=1)
            x_range = np.arange(W)
            fitted_ys = np.polyval(coeffs, x_range)
            fitted_ys = gaussian_filter1d(fitted_ys, sigma=1).astype(np.int32)

            region_vecs = instance_pred[:, ys, xs]
            mean_vec = np.mean(region_vecs, axis=1)

            for x, y in zip(x_range, fitted_ys):
                if 0 <= x < W and 0 <= y < H:
                    for dy in range(-2, 3):
                        py = y + dy
                        if 0 <= py < H:
                            extended_instance[:, py, x] = mean_vec
        except Exception as e:
            print(f"[⚠️ fitting error on label {label}] {e}")
            continue

    return extended_instance

In [20]:
def get_road_position(instance_pred, image_rgb):
    """
    instance_pred: (3, H, W) 차선 인스턴스 마스크
    image_rgb: (H, W, 3) RGB 원본 이미지
    return: 'left', 'center', or 'right'
    """
    instance_map = np.mean(instance_pred, axis=0)
    h, w = instance_map.shape

    # 마지막 행(row) 기준으로만 판단
    bottom_row = instance_map[h - 1, :]
    mask_row = (bottom_row > 30).astype(np.uint8)

    coords_x = np.where(mask_row > 0)[0]  # x 좌표만 추출

    if len(coords_x) == 0:
        return "unknown"

    mean_x = np.mean(coords_x)
    mean_y = h - 1  # 마지막 행

    if mean_x < w / 3:
        direction = "right"
    elif mean_x > w * 2 / 3:
        direction = "left"
    else:
        direction = "center"

    return direction

In [21]:
# 침범 감지 함수 정의
def check_intrusion(track, binary, frame_size):
    x1, y1, x2, y2 = map(int, track.to_ltrb())
    w_frame, h_frame = frame_size
    fc = ((x1 + x2) // 2, y2)
    mx = int(fc[0] * (binary.shape[1] / w_frame))
    my = int(fc[1] * (binary.shape[0] / h_frame))
    mx, my = np.clip(mx, 0, binary.shape[1]-1), np.clip(my, 0, binary.shape[0]-1)
    pixel_val = binary[my, mx]
    on_lane = pixel_val > 128
    return on_lane, fc, (mx, my)

In [22]:
# Camera 기준값
FOCAL_LENGTH = 600
REAL_CAR_WIDTH = 1.3  # m

SAFE_DISTANCE_TABLE = {
    120: 120,
    110: 110,
    100: 100,
    90: 90,
    80: 80,
    70: 70,
    60: 55,
    50: 45,
    40: 25,
    30: 15,
    20: 5
}

def get_recommended_safe_distance(speed_kph: float) -> float:
    for s in sorted(SAFE_DISTANCE_TABLE.keys(), reverse=True):
        if speed_kph >= s:
            return SAFE_DISTANCE_TABLE[s]
    return SAFE_DISTANCE_TABLE[20]

async def process_base64_frame(base64_data, websocket: WebSocket, speed_kph: float):
    print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]}] process start")

    try:
        img_data = base64.b64decode(base64_data)
        nparr = np.frombuffer(img_data, np.uint8)
        frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

        if frame is None:
            print("⚠️ 디코딩 실패")
            return

        rgb_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pil_img = Image.fromarray(rgb_img)
        dummy_input = data_transform(pil_img).to(DEVICE)

        # YOLO
        det = yolo_model.predict(source=[frame], save=False)[0]
        results = []
        for box in det.boxes.data.tolist():
            conf = float(box[4])
            if conf < CONFIDENCE_THRESHOLD:
                continue
            x1, y1, x2, y2 = map(int, box[:4])
            cls = int(box[5])
            results.append([[x1, y1, x2 - x1, y2 - y1], conf, cls])

            if cls in (2, 7):
                car_width = x2 - x1
                if car_width > 0:
                    distance = (REAL_CAR_WIDTH * FOCAL_LENGTH) / car_width
                    recommended = get_recommended_safe_distance(speed_kph)
                    print(f"📏 거리: {distance:.2f}m / 권장: {recommended:.2f}m / 속도: {speed_kph}km/h")

                    if speed_kph > 10:
                        if distance < recommended:
                            payload = {
                                "event": "Distance_Violation",
                                "params": {
                                    "actual_distance": round(distance, 2),
                                    "recommended_distance": recommended
                                }
                            }
                            await websocket.send_text(json.dumps(payload))
                            client_event_counters[websocket]["safe_distance_violation_count"] += 1
                            print(f"⚠️ 안전거리 위반! ({distance:.2f}m < {recommended:.2f}m)")
                    else:
                        # ✅ 정지 시 거리 판단
                        payload = {
                            "event": "Stopped_Distance_Check",
                            "params": {
                                "actual_distance": round(distance, 2)
                            }
                        }
                        await websocket.send_text(json.dumps(payload))
                        client_event_counters[websocket]["safe_distance_violation_count"] += 1
                        print(f"🛑 정지 시 거리 체크: {distance:.2f}m")

        tracks = tracker.update_tracks(results, frame=frame)

        input_tensor = torch.unsqueeze(dummy_input, dim=0)
        outputs = lanenet_model(input_tensor)
        instance_pred = torch.squeeze(outputs['instance_seg_logits'].detach().to('cpu')).numpy() * 255
        binary_pred = torch.squeeze(outputs['binary_seg_pred']).to('cpu').numpy() * 255
        instance_pred = extrapolate_lanes_from_instance(instance_pred)
        binary_pred = (np.mean(instance_pred, axis=0) > 0).astype(np.uint8) * 255

        road_state = get_road_position(instance_pred, cv2.resize(rgb_img, (resize_width, resize_height)))
        print(f"🚗 차량 주행 상태: {road_state}")
        if road_state == "left":
            payload = {"event": "Left_Deviation"}
            await websocket.send_text(json.dumps(payload))
            client_event_counters[websocket]["lane_deviation_left_count"] += 1
        elif road_state == "right":
            payload = {"event": "Right_Deviation"}
            await websocket.send_text(json.dumps(payload))
            client_event_counters[websocket]["lane_deviation_right_count"] += 1

        for track in tracks:
            if not track.is_confirmed():
                continue
            tid = track.track_id
            intrusion, fc, _ = check_intrusion(track, binary_pred, binary_pred.shape)
            if intrusion:
                print(f"🚨 침범 주의: 차량 ID {tid}")
                payload = {"event": "Cut_In"}
                try:
                    await websocket.send_text(json.dumps(payload))
                except Exception as e:
                    print(f"❌ WebSocket send 실패: {e}")

        print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]}] process end")

    except Exception as e:
        print(f"❌ 에러 발생: {e}")

## WebSocket ver.

In [23]:
# WebSocket 핸들러 정의
async def handler(websocket):
    print("✅ 클라이언트 연결됨")
    frame_count = 0
    try:
        while True:
            data = await websocket.recv()
            await process_base64_frame(data)

            frame_count += 1

    except Exception as e:
        print(f"❌ 에러 발생: {e}")
    finally:
        print("🛑 클라이언트 연결 종료")

In [24]:
## WebSocket 서버 정의 및 실행

## Colab에서 asyncio 실행 가능하게 설정
#nest_asyncio.apply()

#async def main():
#    async with websockets.serve(handler, "0.0.0.0", 8080):
#        print("📡 WebSocket 서버 대기 중 (port: 8080)...")
#        await asyncio.Future()  # 영원히 대기

#await main()

## Video ver.

In [25]:
# # 동영상 -> base64 프레임

# # 동영상 파일 불러오기
# cap = cv2.VideoCapture('data/driving_sample.mp4')

# frame_count = 0
# FRAME_SKIP = 10

# while cap.isOpened():
#     ret, frame = cap.read()
#     if not ret:
#         break

#     if frame_count % FRAME_SKIP != 0:
#         frame_count += 1
#         continue

#     _, buffer = cv2.imencode('.jpg', frame)
#     b64 = base64.b64encode(buffer).decode('utf-8')

#     await process_base64_frame(b64)
#     frame_count += 1

# cap.release()
# print("✅ 테스트 종료")

In [26]:
# 이미지 -> base64 프레임

# 이미지 파일 불러오기
#image = cv2.imread('/content/drive/MyDrive/Vroomie/training/val/원천데이터_flat/2_20200731_140619_004560.jpg')
#_, buffer = cv2.imencode('.jpg', image)
#b64 = base64.b64encode(buffer).decode('utf-8')

# from io import BytesIO

# image = Image.open('/content/drive/MyDrive/Vroomie/training/train/원천데이터_flat/2_20200731_141713_000330.jpg')
# buff = BytesIO()
# image.save(buff, format="JPEG")
# b64 = base64.b64encode(buff.getvalue())
#await process_base64_frame(b64)

In [27]:
!ngrok authtoken <token>

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [28]:
# WebSocket FastAPI 서버
from fastapi import FastAPI, WebSocket, WebSocketDisconnect
import nest_asyncio
import uvicorn
from pyngrok import ngrok

# FastAPI 인스턴스 생성
app = FastAPI()

# 클라이언트 추적
connected_clients = set()

# 클라이언트 이벤트 카운터
client_event_counters = {}

# WebSocket 엔드포인트
@app.websocket("/")
async def websocket_endpoint(websocket: WebSocket):
    global colab_ws
    await websocket.accept()
    connected_clients.add(websocket)
    print("✅ WebSocket 연결됨")

    # 클라이언트 별 카운터 초기화
    client_event_counters[websocket] = {
      "lane_deviation_left_count": 0,
      "lane_deviation_right_count": 0,
      "safe_distance_violation_count": 0,
      "sudden_deceleration_count": 0,
      "sudden_acceleration_count": 0,
      "speeding_count": 0
    }

    try:
        while True:
            json_text = await websocket.receive_text()
            data = json.loads(json_text)

            base64_frame = data["frame"]
            speed_kph = data["speed"]

            await process_base64_frame(base64_frame, websocket, speed_kph)

    except WebSocketDisconnect:
        print("❌ WebSocket 연결 종료됨")
        connected_clients.discard(websocket)
    except Exception as e:
        print(f"❌ WebSocket 처리 중 에러: {e}")
        connected_clients.discard(websocket)

# Colab용 이벤트루프 패치
nest_asyncio.apply()

# ngrok으로 포트 열기
public_url = ngrok.connect(8000, "http")
print(f"🔗 외부 접속 URL: {public_url}")

# Uvicorn으로 FastAPI 서버 실행 (백그라운드)
uvicorn.run(app, host="0.0.0.0", port=8000)


🔗 외부 접속 URL: NgrokTunnel: "https://48e13d197d76.ngrok.app" -> "http://localhost:8000"


INFO:     Started server process [10329]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     2406:5900:1154:9012:4d55:27d6:739:d738:0 - "WebSocket /" [accepted]
INFO:     connection open


✅ WebSocket 연결됨
[2025-06-03 14:03:32.960] process start

0: 480x640 1 car, 1 bus, 55.8ms
Speed: 3.9ms preprocess, 55.8ms inference, 136.9ms postprocess per image at shape (1, 3, 480, 640)
📏 거리: 35.45m / 권장: 5.00m / 속도: 0km/h
🛑 정지 시 거리 체크: 35.45m
129599
[2025-06-03 14:03:34.314] [CUML] [warning] Batch size limited by the chosen integer type (4 bytes). 18517 -> 16570. Using the larger integer type might result in better performance
🚗 차량 주행 상태: center
[2025-06-03 14:03:36.101] process end
[2025-06-03 14:03:36.104] process start

0: 480x640 3 cars, 2 buss, 23.8ms
Speed: 1.7ms preprocess, 23.8ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)
📏 거리: 4.04m / 권장: 5.00m / 속도: 0km/h
🛑 정지 시 거리 체크: 4.04m
📏 거리: 37.14m / 권장: 5.00m / 속도: 0km/h
🛑 정지 시 거리 체크: 37.14m
129600
[2025-06-03 14:03:36.215] [CUML] [warning] Batch size limited by the chosen integer type (4 bytes). 18517 -> 16570. Using the larger integer type might result in better performance
🚗 차량 주행 상태: center
[2025-06-03 14:0

INFO:     connection closed
INFO:     2406:5900:1154:9012:4d55:27d6:739:d738:0 - "WebSocket /" [accepted]
INFO:     connection open


📏 거리: 4.56m / 권장: 5.00m / 속도: 2.0034037km/h
❌ 에러 발생: 
❌ WebSocket 처리 중 에러: WebSocket is not connected. Need to call "accept" first.
✅ WebSocket 연결됨
[2025-06-03 14:04:25.904] process start

0: 480x640 1 laptop, 23.9ms
Speed: 1.7ms preprocess, 23.9ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)
129600
[2025-06-03 14:04:25.979] [CUML] [warning] Batch size limited by the chosen integer type (4 bytes). 18517 -> 16570. Using the larger integer type might result in better performance
🚗 차량 주행 상태: unknown
[2025-06-03 14:04:27.392] process end
[2025-06-03 14:04:27.394] process start

0: 480x640 1 laptop, 25.8ms
Speed: 3.4ms preprocess, 25.8ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)
129600
[2025-06-03 14:04:27.470] [CUML] [warning] Batch size limited by the chosen integer type (4 bytes). 18517 -> 16570. Using the larger integer type might result in better performance
🚗 차량 주행 상태: unknown
[2025-06-03 14:04:28.890] process end
[2025-06-03 14:04:28.894] pr

...

INFO:     2406:5900:1154:9012:4d55:27d6:739:d738:0 - "WebSocket /" [accepted]
INFO:     connection open


✅ WebSocket 연결됨


INFO:     connection closed


❌ WebSocket 연결 종료됨


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [10329]
